In [45]:
import sys
import os
from scipy import stats
import itertools
import numpy as np
from colour import Color
import matplotlib.pyplot as plt
from collections import Counter
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns

from matplotlib import collections  as mc

In [46]:
### bokeh imports
from bokeh.io import curdoc, output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select, TextInput, Slider, Button, PreText, HoverTool, Cross, Range1d, Segment, Circle, field
output_notebook()

Loading BokehJS ...

In [52]:
color_companies = {}

In [53]:
def read_input(test_path):
    print(test_path)
    plansza = []
    skills = {}
    companies = {}
    companies_simple = []
    bonuses = []
    nums_skills = []
    mngr_bonuses = []
    skills_by_developer = []
    skills_to_companies = {}
    with open(test_path, "r") as f:
        n, m = [int(a) for a in f.readline().split(" ")]
        for i in range(m):
            plansza.append(f.readline().strip())
        devs = int(f.readline())
        for i in range(devs):
            l = f.readline().strip().split(" ")
            company = l[0]
            companies_simple.append(company)
            if company not in color_companies:
                color_companies[company] = [np.random.randint(0, 190) for _ in range(3)] + [255]
            b = int(l[1])
            bonuses.append(b)
            num_s = int(l[2])
            nums_skills.append(num_s)
            if company in companies:
                companies[company].append(i)
            else:
                companies[company] = [i]
            skills_by_developer.append(l[3:])
            assert(len(l[3:]) == len(set(l[3:])))
            
            for skill in l[3:]:                
                if skill in skills:
                    skills[skill].append(i)
                    skills_to_companies[skill].add(company)
                else:
                    skills[skill] = [i]
                    skills_to_companies[skill] = {company}
        mngrs = int(f.readline())
        for i in range(mngrs):
            company, bonus = f.readline().strip().split(" ")
            bonus = int(bonus)
            companies[company].append(i+devs)
            companies_simple.append(company)
            mngr_bonuses.append(bonus)
            bonuses.append(bonus)
            skills_by_developer.append([])
    return {
        "plansza": plansza,
        "skills": skills,
        "bonuses": bonuses,
        "mngr_bonuses": mngr_bonuses,
        "skills_by_developer": skills_by_developer,
        "skills_to_companies": skills_to_companies,
        "nums_skills": nums_skills,
        "devs": devs,
        "mngrs": mngrs,
        "companies": companies,
        "companies_simple": companies_simple
    }

In [54]:
def read_output(test_path):
    with open(test_path, "r") as f:
        x = f.readlines()
    x = [a.strip().split(" ") for a in x]
    x = [a if len(a) == 2 else [] for a in x]
    x = [[int(b) for b in a] for a in x]
    return {"coordinates" : x}

In [55]:
def get_coor(plansza):
    return len(plansza), len(plansza[0])

def get_placement(INPUT, OUTPUT):
    H, W = get_coor(INPUT["plansza"])
    image = np.zeros(H*W*4)
    image = image.reshape((H, W, 4)) 
    
    x = [[-2 if INPUT["plansza"][i][j] != "#" else -1 for j in range(W)] for i in range(H)]
    for i in range(len(OUTPUT["coordinates"])):
        if len(OUTPUT["coordinates"][i]) > 0:
            a, b = OUTPUT["coordinates"][i]
            x[b][a] = i
    for i in range(H):
        for j in range(W):
            if x[i][j] == -1:
                image[i][j] = np.array([255]*4)
            elif x[i][j] == -2:
                image[i][j] = np.array([240, 240, 240, 255])
            else:
                image[i][j] = np.array(color_companies[INPUT["companies_simple"][x[i][j]]])
    return image

In [134]:
import itertools
flatten = itertools.chain.from_iterable

def get_imaging_data():
    INPUT = read_input("b_dream.txt")
    OUTPUT = read_output("out-16.txt")
    
    fake_image = get_placement(INPUT, OUTPUT)
    
    H, W = get_coor(INPUT["plansza"])
    img = np.empty((H,W), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((H, W, 4))
    for i in range(H):
        for j in range(W):
            view[i, j] = fake_image[i, j]
                
    print(H, W)
    blinkers = [(i, j) for i, j in itertools.product(range(H), range(W)) if INPUT["plansza"][i][j] == "M"]
    
    pixel_data = [[dict(
        x="", 
        y="", 
        hover_xy="",
        hover_company="", 
        hover_cell_type="",
        hover_worker_id="",
        hover_bonus="") for i in range(W)] for j in range(H)]
    
    ct = {
        "_": "dev",
        "#": "empty",
        "M": "manager"
    }
    
    for i in range(H):
        for j in range(W):
            pixel_data[i][j]["x"] = i + 0.5
            pixel_data[i][j]["y"] = j + 0.5
            pixel_data[i][j]["hover_xy"] = (i,j)
            pixel_data[i][j]["hover_cell_type"] = ct[INPUT["plansza"][j][i]]
    
    for i in range(len(OUTPUT["coordinates"])):
        if len(OUTPUT["coordinates"][i]) > 0:
            b, a = OUTPUT["coordinates"][i]
            pixel_data[b][a]["hover_worker_id"] = i
            pixel_data[b][a]["hover_company"] = INPUT["companies_simple"][i]
            pixel_data[b][a]["hover_bonus"] = INPUT["bonuses"][i]
            
    
    LD = list(flatten(pixel_data))
    #print(LD)
    pixels = {k: [dic[k] for dic in LD] for k in LD[0]}
    
    workers = [[-2 if INPUT["plansza"][i][j] != "#" else -1 for j in range(W)] for i in range(H)]
    for i in range(len(OUTPUT["coordinates"])):
        if len(OUTPUT["coordinates"][i]) > 0:
            a, b = OUTPUT["coordinates"][i]
            workers[b][a] = i
    
    return dict(
        H=H,
        W=W,
        img=img,
        img_view=view,
        blinkers=blinkers,
        pixels=pixels,
        INPUT=INPUT,
        OUTPUT=OUTPUT,
        workers=workers,
    )

In [142]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ImageRGBA, Rect
from bokeh.events import Tap

def modify_doc(doc):
    data = get_imaging_data()
    
    assert ("img" in data)
    assert ("img_view" in data)

    p = figure(tools="pan,wheel_zoom,box_zoom,reset", 
               toolbar_location="below", 
                title=None,
                plot_width=900,
                plot_height=900,
                x_range=Range1d(-1, data["W"]),
                y_range=Range1d(-1, data["H"]),
    )
    p.xaxis.axis_line_width = 0
    p.yaxis.axis_line_width = 0
    p.xaxis.major_tick_in = 0
    p.yaxis.major_tick_in = 0
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #p.x_range.range_padding = p.y_range.range_padding = 0

    sources = dict()
    
    ### add image glyph
    sources["image"] = ColumnDataSource(data={"image": [data["img"]]})    
    image_glyph = ImageRGBA(image="image", x=0, y=0, dw=data["W"], dh=data["H"])
    p.add_glyph(sources["image"], image_glyph)

    ### add invisible squares with hovers over pixels
    inv_squares_glyph = Rect(x="x",y="y", width=1, height=1, line_alpha=0.0, fill_alpha=0.0)
    sources["pixels"] = ColumnDataSource(data=data["pixels"])
    hover_shit_happening_here = p.add_glyph(sources["pixels"], inv_squares_glyph)
    TOOLTIPS = [
    ]
    TOOLTIPS += [(hover_token[6:], "@" + hover_token) for hover_token in sources["pixels"].data if hover_token[:6] == "hover_"]
    hover = HoverTool(renderers=[hover_shit_happening_here], tooltips=TOOLTIPS)
    p.add_tools(hover)
    
    ### add blinking animation for selected fields
    if "blinkers" in data:
        def animate_blinking_pixels():
            dimming_order = [255, 180, 100, 170, 255]
            for x, y in data["blinkers"]:
                try:
                    pos = dimming_order.index(data["img_view"][x, y, 3])
                    data["img_view"][x, y, 3] = dimming_order[pos + 1]
                except:
                    pass
            sources["image"].data={"image": [data["img"]]}

        doc.add_periodic_callback(animate_blinking_pixels, period_milliseconds=200)
    
    ### add onclick logic for the plot
    tap_event_data = dict(is_clicked=False, company="ALL")
    
    def just_fucking_update():
        for x in range(data["W"]):
            for y in range(data["H"]):
                if tap_event_data["is_clicked"] is False:
                    data["img_view"][x, y, 3] = 255
                else:
                    if data["workers"][x][y] >= 0 and data["INPUT"]["companies_simple"][data["workers"][x][y]] == tap_event_data["company"]:
                        data["img_view"][x, y, 3] = 255
                    else:
                        data["img_view"][x, y, 3] = 0
        
        sources["image"].data={"image": [data["img"]]}
        return
                        
    
    def callback(event):
        if tap_event_data["is_clicked"] is not False:
            tap_event_data["is_clicked"] = False
            tap_event_data["company"] = "ALL"
            just_fucking_update()
            return
        x, y = int(event.x),int(event.y)
        if x < 0 or x >= data["W"] or y < 0 or y >= data["H"]:
            return
        if data["workers"][y][x] < 0:
            return
        tap_event_data["is_clicked"] = True
        tap_event_data["company"] = data["INPUT"]["companies_simple"][data["workers"][y][x]]
        just_fucking_update()
        
    p.on_event(Tap, callback)

    doc.add_root(p)

show(modify_doc)

b_dream.txt
100 100


Dodać:

- żeby po kliknięciu na pole pokazały się wszystkie pola z tej firmy i może jakieś debug data obok, w stylu "firma: twojastara or all, wykorzystane x out of y dewów, x ot of y managerów, score z firmy"